Jupyter notebook to calcualte the Intersection over Union between the groundtruth, 
SHAPR prediction, cylinder and ellipse fit. 
Written by Dominik Waibel & Niklas Kiermeyer

Required Folder Structure

-- SHAPR_dataset
-- -- Ellipse_fit
-- -- Cylinder_fit 

-- SHAPR_results

In [ ]:
#import dependencies
import os 
import numpy as np
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
from skimage import measure
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import trimesh
import cv2
import math
from skimage.transform import resize
import numpy.linalg as linalg
from pyellipsoid import drawing
from skimage.filters import threshold_otsu
from skimage.feature import shape_index
from scipy.ndimage.measurements import center_of_mass
from skimage.filters import gaussian
import seaborn as sns
import keras.backend as K
from scipy.ndimage.morphology import binary_dilation
from scipy.ndimage import gaussian_filter
import copy
from scipy.ndimage.morphology import binary_fill_holes

In [ ]:
# set the path to the dataset folder and SHAPR results
org_path = "./SHAPR_dataset/SHAPR_results/"
test_path = "./SHAPR_dataset//"


In [ ]:
# normalize and threshold the data using Otsu's method: 
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html
def norm_thres(data): 
    maxd = np.max(data)
    data = np.nan_to_num(data / maxd)
    if np.max(data)  > 0:
        thresh = threshold_otsu(data)
        #binary = data > 0.5
        #print("threshold", thresh)
        binary = data > thresh
    else: 
        binary = data
    return binary*1.0

In [ ]:
def IoU(y_true,y_pred): 
    intersection = y_true + y_pred
    intersection = np.count_nonzero(intersection > 1.5)
    union = y_true + y_pred
    union = np.count_nonzero(union > 0.5)
    return intersection / union

In [ ]:
IoU_cylinder = []; IoU_ellipse = []; IoU_shapr = []

files = os.listdir(org_path)
print("found", len(files), "files")


for index, file in enumerate(files): 
    print(index, file)
    cylinder3d = imread(test_path + "/Cylinder_fit/"+file)/255.
    ellipse3d = imread(test_path + "/Ellipse_fit/"+file)/255.
    obj_gt = norm_thres(np.nan_to_num(imread(test_path + "obj/"+ file))) 
    SHAPR = norm_thres(np.nan_to_num(np.squeeze(imread(org_path + file))))

    IoU_cylinder.append(IoU(cylinder3d,obj_gt))
    IoU_ellipse.append(IoU(ellipse3d,obj_gt))
    IoU_shapr.append(IoU(SHAPR, obj_gt))
    fname.append(file)


In [ ]:
print("Errors in %")
print(np.mean(IoU_shapr)*100, np.std(IoU_shapr)*100)
print(np.mean(IoU_cylinder)*100,np.std(IoU_cylinder)*100 )
print(np.mean(IoU_ellipse)*100, np.std(IoU_ellipse)*100)

from scipy.stats import wilcoxon

print(wilcoxon(IoU_shapr, IoU_mask3d))
print(wilcoxon(IoU_shapr, IoU_ellipse))

plt.figure(figsize =(8,6))
ax = sns.violinplot(data = [IoU_shapr, IoU_cylinder, IoU_ellipse], 
            showfliers=False,color='lightgray', boxprops={'facecolor':'None'}, orient = "h")
ax = sns.swarmplot(data = [IoU_shapr, IoU_cylinder, IoU_ellipse],color=".25", size =1.5, orient = "h")
plt.xlabel('Wrongly labeled voxels', size = 15)
plt.yticks([0, 1, 2], ["SHAPR","Cylinder",
                       "Ellipse"], size = 15)


plt.locator_params(axis='x', nbins=4)
plt.tight_layout()
plt.grid(b=None)
plt.grid(b=None)

plt.show()